In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [5]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 111 ms


In [6]:
#importing in all clinical_variable files
date= '22112018'
os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/%s_longdf_preImp.csv' %(date),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]

time: 10.5 s


In [20]:
big_df.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
0,200012.0,2153-12-23,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,nit,28448
1,200012.0,2153-12-23,None,vent_recieved,mech/O2/none category,0 days 00:00:00.000000000,ventcategory,28448
2,200012.0,2153-12-23,1,daily_sofa,daily_sofa_score,0 days 00:00:00.000000000,sofa,28448
3,200012.0,2153-12-23,0.0,dobutamine,y/n,0 days 00:00:00.000000000,dobutamine,28448
4,200012.0,2153-12-23,0.0,dopamine,y/n,0 days 00:00:00.000000000,dopamine,28448


time: 11.3 ms


In [27]:
big_df.groupby('label')['value'].describe().sort_values('unique')
##categorical variables:
#rrt, phenylephrine, norepinephrine, vasopressin, dobutamine, dopamine, epinephrine, leukocyte, vent_recieved


,count,unique,top,freq
label,,,,
rrt,14668,2,0.0,13448
phenylephrine,28124,2,1.0,15776
norepinephrine,35398,2,1.0,23637
vasopressin,18146,2,0.0,14240
dobutamine,16003,2,0.0,14430
dopamine,19501,2,0.0,13803
epinephrine,16414,2,0.0,14363
leukocyte,16644,2,Neg/Not_tested,14336
vent_recieved,14668,3,None,8387


time: 1.87 s


In [29]:
big_df[big_df['label']=='vent_recieved']['value'].value_counts() #
df["B"] = df["A"].astype('category')

None      8387
Oxygen    3986
Mech      2295
Name: value, dtype: int64

time: 485 ms


In [31]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')

time: 1.39 s


In [39]:
#big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 1.38 s


In [43]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].describe()

count     14668
unique        3
top        None
freq       8387
Name: value, dtype: object

time: 452 ms


In [232]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine']

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp',
'pao2/fio2']

both_value=['calcium',
'sodium',
'wbc']

time: 6.52 ms


In [190]:
#lambda x: x if not np.isnan(x) else -1

dict1={str(x):max for x in hi_value}
dict2={str(x):min for x in low_value } #dict2={str(x):min for x in low_value }
dict3={str(x):[max,min] for x in both_value }

SyntaxError: invalid syntax (<ipython-input-190-464303920ed4>, line 3)

In [138]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

dict1 = merge_two_dicts(dict1, dict2)
dict1 = merge_two_dicts(dict1, dict3)

time: 4.56 ms


In [173]:
len(dict1.keys())

35

time: 2.23 ms


### to do: need to convert all dtypes to a numeric type taht pereserves nan. 

In [172]:
categorical=['leukocyte', 'vent_recieved']

big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:]
big_noCat= big_df.loc[big_df.loc[:,'label'].isin(dict1.keys()),:]
big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 


time: 1min 45s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [233]:
big_max= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 47.5 s


In [234]:
big_min= big_df.loc[big_df.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values='value', columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 56.7 s


In [235]:
big_both= big_df.loc[big_df.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values='value', columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)

time: 11.4 s


In [196]:
print(len(table),len(table2),len(table3))

14668 14668 14618
time: 1.25 ms


In [236]:
test=pd.merge(table.reset_index(), table2.reset_index(), how='left')
test=pd.merge(test, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')

time: 67.5 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [237]:
print(len(list(test)), #40- icustay_id - label = 38. with 

big_df['label'].nunique(),
     
set(big_df['label'].unique())- set(test)   


41 38
time: 325 ms


In [ ]:
big_max= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]

In [245]:
big_categorical.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
0,200012.0,2153-12-23,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,nit,28448
1,200012.0,2153-12-23,None,vent_recieved,mech/O2/none category,0 days 00:00:00.000000000,ventcategory,28448
84,200014.0,2105-02-17,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,nit,9514
85,200014.0,2105-02-17,Mech,vent_recieved,mech/O2/none category,0 days 00:00:00.000000000,ventcategory,9514
471,200019.0,2178-07-09,Neg/Not_tested,leukocyte,pos/neg category,0 days 00:00:00.000000000,nit,21789


time: 10.1 ms


In [270]:
big_categorical[(big_categorical["label"]=='leukocyte') & (big_categorical["icustay_id"].isin(icustay_id))]
#16644

# big_categorical[big_categorical[['icustay_id','label']].duplicated(keep=False)]

,icustay_id,t_0,value,label,uom,delta,source,subject_id
10960,200140.0,2197-01-18,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,27874
12794,200143.0,2191-04-03,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,94414
16234,200220.0,2148-01-07,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,13960
17394,200240.0,2147-10-26,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,76327
19972,200282.0,2164-05-03,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,77484
22525,200301.0,2167-06-25,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,9374
23319,200318.0,2189-06-06,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,24589
23730,200322.0,2168-04-21,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,29825
25609,200339.0,2192-02-23,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,51839
29087,200352.0,2171-09-28,pos,leukocyte,pos/neg category,0 days 00:00:00.000000000,leuk,97181


time: 37.2 ms


In [265]:
#big_categorical[big_categorical['icustay_id']==200140.0]
#big_categorical.loc[big_categorical['source']=='nit','label']="nitrite" #fixing a bug i foun in 03.1 clinical var prep. has been fixed since. 


time: 314 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [252]:
big_categorical["icustay_id"].isin(icustay_id)

0          True
1          True
84         True
85         True
471        True
472        True
813        True
814        True
1439       True
1440       True
2068       True
2069       True
2458       True
2459       True
2685       True
2686       True
3900       True
3901       True
4219       True
4220       True
5004       True
5005       True
5924       True
5925       True
6151       True
6152       True
6327       True
6328       True
6933       True
6934       True
           ... 
7196729    True
7196878    True
7196879    True
7197709    True
7197710    True
7198191    True
7198192    True
7198360    True
7198361    True
7198662    True
7198663    True
7198664    True
7199118    True
7199119    True
7199527    True
7199528    True
7199838    True
7199839    True
7200121    True
7200122    True
7200991    True
7200992    True
7201970    True
7201971    True
7202726    True
7202727    True
7203649    True
7203650    True
7204601    True
7204602    True
Name: icustay_id, Length

time: 6.69 ms


In [247]:
big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label') #need to convert to long format. should be one row per patient per time. 

ValueError: Index contains duplicate entries, cannot reshape

time: 23.2 ms


In [105]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

KeyboardInterrupt: 

time: 57.3 s


In [187]:
test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


label,bands,bicarbonate,bilirubin,bun,calcium,chloride,creatinine,daily_sofa,diasbp,dobutamine,...,ptt,resprate,rrt,sodium,spo2,sysbp,temperature,vasopressin,wbc,weight
icustay_id,,,,,,,,,,,,,,,,,,,,,
200012.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,82.0,0.0,...,NaN,27.0,0.0,NaN,99.0,109.0,37.500000,0.0,4.9,51.20
200014.0,NaN,28.0,NaN,24.0,1.24,114.0,0.8,2.0,93.0,0.0,...,30.1,33.0,0.0,143.0,100.0,178.0,37.000000,0.0,14.3,62.00
200019.0,NaN,21.0,NaN,24.0,NaN,109.0,1.0,6.0,90.0,0.0,...,30.9,29.0,0.0,140.0,100.0,206.0,37.777778,0.0,13.5,65.00
200030.0,NaN,19.0,14.5,28.0,1.33,111.0,1.0,4.0,104.0,1.0,...,30.8,33.0,0.0,141.0,99.0,182.0,38.100000,0.0,13.9,113.60
200033.0,NaN,27.0,NaN,16.0,1.06,108.0,0.7,1.0,98.0,0.0,...,29.7,36.0,0.0,145.0,100.0,158.0,38.611111,0.0,7.7,74.00
200036.0,NaN,23.0,NaN,31.0,NaN,110.0,1.0,1.0,111.0,0.0,...,44.2,34.0,0.0,141.0,100.0,155.0,37.333332,0.0,6.7,79.00
200055.0,NaN,30.0,0.3,40.0,1.11,100.0,8.9,8.0,75.0,0.0,...,31.2,30.0,1.0,140.0,100.0,118.0,36.277778,0.0,6.3,56.00
200059.0,NaN,30.0,0.7,50.0,1.17,109.0,1.5,6.0,100.0,0.0,...,141.7,35.0,0.0,140.0,100.0,191.0,37.400002,0.0,14.8,54.00
200061.0,NaN,27.0,5.9,15.0,NaN,106.0,0.9,3.0,95.0,0.0,...,NaN,31.0,0.0,142.0,97.0,162.0,38.944444,0.0,15.6,250.00


time: 2.45 s


In [177]:
table

label,bands,bicarbonate,bilirubin,bun,calcium,chloride,creatinine,daily_sofa,diasbp,dobutamine,...,ptt,resprate,rrt,sodium,spo2,sysbp,temperature,vasopressin,wbc,weight
icustay_id,,,,,,,,,,,,,,,,,,,,,
200012.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,82.0,0.0,...,NaN,27.0,0.0,NaN,99.0,109.0,37.500000,0.0,4.9,51.20
200014.0,NaN,28.0,NaN,24.0,1.24,114.0,0.8,2.0,93.0,0.0,...,30.1,33.0,0.0,143.0,100.0,178.0,37.000000,0.0,14.3,62.00
200019.0,NaN,21.0,NaN,24.0,NaN,109.0,1.0,6.0,90.0,0.0,...,30.9,29.0,0.0,140.0,100.0,206.0,37.777778,0.0,13.5,65.00
200030.0,NaN,19.0,14.5,28.0,1.33,111.0,1.0,4.0,104.0,1.0,...,30.8,33.0,0.0,141.0,99.0,182.0,38.100000,0.0,13.9,113.60
200033.0,NaN,27.0,NaN,16.0,1.06,108.0,0.7,1.0,98.0,0.0,...,29.7,36.0,0.0,145.0,100.0,158.0,38.611111,0.0,7.7,74.00
200036.0,NaN,23.0,NaN,31.0,NaN,110.0,1.0,1.0,111.0,0.0,...,44.2,34.0,0.0,141.0,100.0,155.0,37.333332,0.0,6.7,79.00
200055.0,NaN,30.0,0.3,40.0,1.11,100.0,8.9,8.0,75.0,0.0,...,31.2,30.0,1.0,140.0,100.0,118.0,36.277778,0.0,6.3,56.00
200059.0,NaN,30.0,0.7,50.0,1.17,109.0,1.5,6.0,100.0,0.0,...,141.7,35.0,0.0,140.0,100.0,191.0,37.400002,0.0,14.8,54.00
200061.0,NaN,27.0,5.9,15.0,NaN,106.0,0.9,3.0,95.0,0.0,...,NaN,31.0,0.0,142.0,97.0,162.0,38.944444,0.0,15.6,250.00


time: 78.9 ms


In [174]:
test.head()

icustay_id  bands bilirubin    bun chloride creatinine daily_sofa glucose  \
                max       max    max      max        max        max     max   
0   200012.0  129.0     129.0  129.0    129.0      129.0      129.0   129.0   
1   200014.0  178.0     178.0  178.0    178.0      178.0      178.0   178.0   
2   200019.0  206.0     206.0  206.0    206.0      206.0      206.0   206.0   
3   200030.0  270.0     270.0  270.0    270.0      270.0      270.0   270.0   
4   200033.0  326.0     326.0  326.0    326.0      326.0      326.0   326.0   

  heartrate    inr ...    ph platelet spo2 sysbp calcium      sodium       \
        max    max ...   min      min  min   min     max  min    max  min   
0     129.0  129.0 ...   0.0      0.0  0.0   0.0   129.0  0.0  129.0  0.0   
1     178.0  178.0 ...   0.0      0.0  0.0   0.0   178.0  0.0  178.0  0.0   
2     206.0  206.0 ...   0.0      0.0  0.0   0.0   206.0  0.0  206.0  0.0   
3     270.0  270.0 ...   0.0      0.0  0.0   0.0   270.0  0.0  270.0  0.0   
4     326.0  326.0 ...   0.0      0.0  0.0   0.0   326.0  0.0  326.0  0.0   

     wbc       
     max  min  
0  129.0  0.0  
1  178.0  0.0  
2  206.0  0.0  
3  270.0  0.0  
4  326.0  0.0  

[5 rows x 39 columns]

time: 35.6 ms


In [117]:
big_noCat.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
2,200012.0,2153-12-23,1.0,daily_sofa,daily_sofa_score,0 days 00:00:00.000000000,sofa,28448
3,200012.0,2153-12-23,0.0,dobutamine,y/n,0 days 00:00:00.000000000,dobutamine,28448
4,200012.0,2153-12-23,0.0,dopamine,y/n,0 days 00:00:00.000000000,dopamine,28448
5,200012.0,2153-12-23,0.0,epinephrine,y/n,0 days 00:00:00.000000000,epinephrine,28448
6,200012.0,2153-12-23,0.0,norepinephrine,y/n,0 days 00:00:00.000000000,norepinephrine,28448


time: 11.5 ms


In [175]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


KeyError: "Column 'bands' does not exist!"

time: 180 ms


In [141]:
#big_df['label'].unique() #bands is in this
big_noCat['label'].unique()

array(['daily_sofa', 'dobutamine', 'dopamine', 'epinephrine',
       'norepinephrine', 'phenylephrine', 'rrt', 'vasopressin', 'weight',
       'lactate', 'diasbp', 'heartrate', 'meanartpress', 'resprate',
       'spo2', 'sysbp', 'temperature', 'bands', 'hemoglobin', 'platelet',
       'wbc', 'mingcs', 'calcium', 'glucose', 'pco2', 'ph', 'bicarbonate',
       'bun', 'chloride', 'creatinine', 'inr', 'potassium', 'ptt',
       'sodium', 'bilirubin'], dtype=object)

time: 284 ms


In [170]:
test.loc[test.loc[:,'icustay_id']==200946,'ph']

#test # ph needs to be null

,min
147,0.0


time: 7.44 ms


In [180]:
big_noCat[big_noCat['value'].isnull()].head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
80023,200946.0,2117-05-21,NaN,pco2,mm Hg,0 days 05:16:00.000000000,bg_ART,91399
80024,200946.0,2117-05-21,NaN,ph,units,0 days 05:16:00.000000000,bg_all,91399
190455,202256.0,2120-09-14,NaN,calcium,mmol/L,1 days 08:04:00.000000000,bg_all,32504
190456,202256.0,2120-09-14,NaN,lactate,mmol/L,1 days 08:04:00.000000000,bg_all,32504
190457,202256.0,2120-09-14,NaN,pco2,mm Hg,1 days 08:04:00.000000000,bg_ART,32504


time: 457 ms


In [55]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

time: 539 ms


In [66]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

time: 623 ms


In [179]:
%whos

# del big_agg
# del hi_df
# del final_pt_df2

Variable          Type                Data/Info
-----------------------------------------------
ConfigManager     MetaHasTraits       <class 'notebook.services<...>g.manager.ConfigManager'>
Memory            type                <class 'sklearn.externals.joblib.memory.Memory'>
Path              type                <class 'pathlib.Path'>
big_agg           DataFrameGroupBy    <pandas.core.groupby.grou<...>y object at 0x1a13017320>
big_categorical   DataFrame                    icustay_id      <...>n[31312 rows x 8 columns]
big_df            DataFrame                    icustay_id      <...>7204842 rows x 8 columns]
big_noCat         DataFrame                    icustay_id      <...>7158862 rows x 8 columns]
both_value        list                n=3
categorical       list                n=2
cm                ConfigManager       <notebook.services.config<...>r object at 0x1a1273e320>
date              str                 22112018
dict1             dict                n=35
dict2             